In [1]:
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

subprocess.run(
  ["git", "clone", REPO_URL, str(REPO_DIR)],
  check=True
)

USE_CLEAN = False  # <--- Set True to remove existing environment
opts = "--clean" if USE_CLEAN else ""

get_ipython().run_line_magic(
    "run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts}"
)

🔧 FEniCSx Setup Configuration
PETSc type      : real
Clean install   : False

⚠️  Google Drive not mounted — using local cache (/content)

🔧 Installing FEniCSx environment...

🔍 Verifying PETSc type...
✅ Installed: Real PETSc (float64)

✨ Loading FEniCSx Jupyter magic... %%fenicsx registered

✅ FEniCSx setup complete!

Next steps:
  1. Run %%fenicsx --info to verify installation
  2. Use %%fenicsx in cells to run FEniCSx code
  3. Use -np N for parallel execution (e.g., %%fenicsx -np 4)

📌 Note: Real PETSc is installed
   - Recommended for most FEM problems
   - For complex problems, reinstall with --complex


---

In [2]:
%%fenicsx -np 4

import numpy as np
from mpi4py import MPI

import pyvista as pv

from dolfinx import mesh, fem, plot
import ufl

# Get MPI information
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Create a simple 2D mesh (unit square with quadrilaterals)
# Mesh will be automatically partitioned across MPI processes
domain = mesh.create_unit_square(comm, 8, 8, mesh.CellType.quadrilateral)

# Print MPI partition information
print(f"\n{'='*70}")
print(f"MPI Process {rank}/{size}")
print(f"{'='*70}")

# Print mesh distribution information
print(f"\nMesh Distribution on Rank {rank}:")
print(f"  - Local cells: {domain.topology.index_map(domain.topology.dim).size_local}")
print(f"  - Ghost cells: {domain.topology.index_map(domain.topology.dim).num_ghosts}")
print(f"  - Local vertices: {domain.topology.index_map(0).size_local}")
print(f"  - Ghost vertices: {domain.topology.index_map(0).num_ghosts}")

# Demonstrate various finite element spaces with different degrees
print(f"\nDOLFINx Function Space Examples on Rank {rank}:")
print("=" * 70)

# 1. Continuous Lagrange elements (CG) - different degrees
print(f"\n1. Lagrange P1 (degree 1):")
V1 = fem.functionspace(domain, ("Lagrange", 1))
print(f"   - DOFs per cell: {V1.element.space_dimension}")
print(f"   - Local DOFs: {V1.dofmap.index_map.size_local}")
print(f"   - Ghost DOFs: {V1.dofmap.index_map.num_ghosts}")
print(f"   - Total DOFs (local + ghost): {V1.dofmap.index_map.size_local + V1.dofmap.index_map.num_ghosts}")

print(f"\n2. Lagrange P2 (degree 2):")
V2 = fem.functionspace(domain, ("Lagrange", 2))
print(f"   - DOFs per cell: {V2.element.space_dimension}")
print(f"   - Local DOFs: {V2.dofmap.index_map.size_local}")
print(f"   - Ghost DOFs: {V2.dofmap.index_map.num_ghosts}")

print(f"\n3. Lagrange P3 (degree 3):")
V3 = fem.functionspace(domain, ("Lagrange", 3))
print(f"   - DOFs per cell: {V3.element.space_dimension}")
print(f"   - Local DOFs: {V3.dofmap.index_map.size_local}")
print(f"   - Ghost DOFs: {V3.dofmap.index_map.num_ghosts}")

print(f"\n4. Discontinuous Galerkin (DG) elements:")
V_dg0 = fem.functionspace(domain, ("DG", 0))  # Piecewise constant
V_dg1 = fem.functionspace(domain, ("DG", 1))  # Discontinuous P1
print(f"   DG0 Local DOFs: {V_dg0.dofmap.index_map.size_local}")
print(f"   DG0 Ghost DOFs: {V_dg0.dofmap.index_map.num_ghosts}")
print(f"   DG1 Local DOFs: {V_dg1.dofmap.index_map.size_local}")
print(f"   DG1 Ghost DOFs: {V_dg1.dofmap.index_map.num_ghosts}")
print(f"   Note: DG elements typically have more ghost DOFs due to discontinuity")

print(f"\n5. Vector Lagrange elements (for vector fields):")
V_vec = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim,)))
print(f"   Local DOFs: {V_vec.dofmap.index_map.size_local}")
print(f"   Ghost DOFs: {V_vec.dofmap.index_map.num_ghosts}")
print(f"   Components per node: {domain.geometry.dim}")

# Detailed ghost information for P1 space
print(f"\n6. Detailed Ghost Map Information (P1 space):")
print("-" * 70)
dofmap1 = V1.dofmap
index_map = dofmap1.index_map

print(f"   Index Map Information:")
print(f"   - Size local: {index_map.size_local}")
print(f"   - Num ghosts: {index_map.num_ghosts}")
print(f"   - Size global: {index_map.size_global}")
print(f"   - Local range: [{index_map.local_range[0]}, {index_map.local_range[1]})")

# Get ghost owners (which rank owns each ghost DOF)
if index_map.num_ghosts > 0:
    ghost_owners = index_map.owners
    print(f"   - Ghost owners (first 10): {ghost_owners[:min(10, len(ghost_owners))]}")
else:
    print(f"   - No ghost DOFs on this rank")

# Print cell distribution
print(f"\n7. Cell Distribution Across Processes:")
print("-" * 70)
cell_map = domain.topology.index_map(domain.topology.dim)
print(f"   Rank {rank}:")
print(f"   - Owns cells: [{cell_map.local_range[0]}, {cell_map.local_range[1]})")
print(f"   - Local cells: {cell_map.size_local}")
print(f"   - Ghost cells: {cell_map.num_ghosts}")
print(f"   - Total global cells: {cell_map.size_global}")

# Gather statistics from all processes
local_cells = cell_map.size_local
local_dofs = V1.dofmap.index_map.size_local
ghost_dofs = V1.dofmap.index_map.num_ghosts

all_local_cells = comm.gather(local_cells, root=0)
all_local_dofs = comm.gather(local_dofs, root=0)
all_ghost_dofs = comm.gather(ghost_dofs, root=0)

if rank == 0:
    print(f"\n{'='*70}")
    print(f"Summary Across All {size} Processes:")
    print(f"{'='*70}")
    print(f"\nCells per process: {all_local_cells}")
    print(f"Local DOFs per process (P1): {all_local_dofs}")
    print(f"Ghost DOFs per process (P1): {all_ghost_dofs}")
    print(f"\nTotal cells: {sum(all_local_cells)}")
    print(f"Total global DOFs: {V1.dofmap.index_map.size_global}")
    print(f"Average ghost DOFs per process: {sum(all_ghost_dofs) / size:.1f}")

# Save mesh partition visualization (ALL RANKS)
topology, cell_types, geometry = plot.vtk_mesh(domain, domain.topology.dim)
grid = pv.UnstructuredGrid(topology, cell_types, geometry)

# Add rank information as cell data
cell_ranks = np.full(cell_map.size_local, rank)
grid.cell_data["MPI_Rank"] = cell_ranks

# Add ghost cell indicator
is_ghost = np.zeros(cell_map.size_local + cell_map.num_ghosts, dtype=np.int32)
is_ghost[cell_map.size_local:] = 1  # Mark ghost cells
grid.cell_data["Is_Ghost"] = is_ghost[:cell_map.size_local]

# Save file for each rank
grid.save(f"mesh_partition_rank_{rank}.vtk")
print(f"\nRank {rank}: Mesh partition saved as mesh_partition_rank_{rank}.vtk")

# Create a function showing DOF ownership
u = fem.Function(V1)
u.x.array[:index_map.size_local] = rank  # Local DOFs get rank value
u.x.array[index_map.size_local:] = -1    # Ghost DOFs get -1

# Save DOF distribution
topology_dof, cell_types_dof, geometry_dof = plot.vtk_mesh(V1)
grid_dof = pv.UnstructuredGrid(topology_dof, cell_types_dof, geometry_dof)
grid_dof.point_data["DOF_Owner"] = u.x.array
grid_dof.save(f"dof_partition_rank_{rank}.vtk")
print(f"Rank {rank}: DOF partition saved as dof_partition_rank_{rank}.vtk")

comm.Barrier()

if rank == 0:
    print(f"\n{'='*70}")
    print("Files saved from all processes:")
    for r in range(size):
        print(f"  - mesh_partition_rank_{r}.vtk")
        print(f"  - dof_partition_rank_{r}.vtk")
    print(f"\n{'='*70}")
    print("Key Observations:")
    print("  - Mesh is partitioned across MPI processes")
    print("  - Each process owns a subset of cells and DOFs")
    print("  - Ghost cells/DOFs are copies from neighboring processes")
    print("  - Continuous elements (CG) share DOFs at boundaries")
    print("  - Discontinuous elements (DG) don't share DOFs")
    print("  - Ghost regions enable communication between processes")
    print(f"{'='*70}")


MPI Process 0/4

Mesh Distribution on Rank 0:
  - Local cells: 16
  - Ghost cells: 8
  - Local vertices: 20
  - Ghost vertices: 15

DOLFINx Function Space Examples on Rank 0:

1. Lagrange P1 (degree 1):
   - DOFs per cell: 4
   - Local DOFs: 20
   - Ghost DOFs: 15
   - Total DOFs (local + ghost): 35

2. Lagrange P2 (degree 2):
   - DOFs per cell: 9
   - Local DOFs: 69
   - Ghost DOFs: 48

3. Lagrange P3 (degree 3):
   - DOFs per cell: 16
   - Local DOFs: 150
   - Ghost DOFs: 97

4. Discontinuous Galerkin (DG) elements:
   DG0 Local DOFs: 16
   DG0 Ghost DOFs: 8
   DG1 Local DOFs: 64
   DG1 Ghost DOFs: 32
   Note: DG elements typically have more ghost DOFs due to discontinuity

5. Vector Lagrange elements (for vector fields):
   Local DOFs: 20
   Ghost DOFs: 15
   Components per node: 2

6. Detailed Ghost Map Information (P1 space):
----------------------------------------------------------------------
   Index Map Information:
   - Size local: 20
   - Num ghosts: 15
   - Size global: 

In [3]:
%%fenicsx -np 4

import numpy as np
from mpi4py import MPI

import pyvista as pv

from dolfinx import mesh, fem, plot
import ufl

# Get MPI information
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Create a 2D mesh (unit square) - automatically partitioned across processes
domain = mesh.create_unit_square(comm, 10, 10, mesh.CellType.triangle)

print(f"\n{'='*70}")
print(f"MPI Process {rank}/{size}")
print(f"{'='*70}")

# Create different function spaces with different degrees
V1 = fem.functionspace(domain, ("Lagrange", 1))  # P1 (linear)
V2 = fem.functionspace(domain, ("Lagrange", 2))  # P2 (quadratic)
V3 = fem.functionspace(domain, ("Lagrange", 3))  # P3 (cubic)

print(f"\nDOLFINx Function Interpolation Examples on Rank {rank}:")
print("=" * 70)

# Print partition information
print(f"\nMesh partition on Rank {rank}:")
print(f"  - Local cells: {domain.topology.index_map(domain.topology.dim).size_local}")
print(f"  - Ghost cells: {domain.topology.index_map(domain.topology.dim).num_ghosts}")
print(f"  - V1 local DOFs: {V1.dofmap.index_map.size_local}")
print(f"  - V1 ghost DOFs: {V1.dofmap.index_map.num_ghosts}")
print(f"  - V2 local DOFs: {V2.dofmap.index_map.size_local}")
print(f"  - V2 ghost DOFs: {V2.dofmap.index_map.num_ghosts}")
print(f"  - V3 local DOFs: {V3.dofmap.index_map.size_local}")
print(f"  - V3 ghost DOFs: {V3.dofmap.index_map.num_ghosts}")

# Example 1: Interpolate an analytical function into a function space
print(f"\n1. Interpolating analytical function f(x,y) = sin(pi*x)*sin(pi*y)")

# Define analytical function
def analytical_function(x):
    return np.sin(np.pi * x[0]) * np.sin(np.pi * x[1])

# Interpolate into P1 space
u1 = fem.Function(V1)
u1.interpolate(analytical_function)
print(f"   Rank {rank}: Interpolated into P1 space (Local DOFs: {V1.dofmap.index_map.size_local})")

# Interpolate into P2 space
u2 = fem.Function(V2)
u2.interpolate(analytical_function)
print(f"   Rank {rank}: Interpolated into P2 space (Local DOFs: {V2.dofmap.index_map.size_local})")

# Interpolate into P3 space
u3 = fem.Function(V3)
u3.interpolate(analytical_function)
print(f"   Rank {rank}: Interpolated into P3 space (Local DOFs: {V3.dofmap.index_map.size_local})")

# Example 2: Interpolate from one space to another
print(f"\n2. Interpolating from P3 space to P1 space")
u1_from_u3 = fem.Function(V1)
u1_from_u3.interpolate(u3)
print(f"   Rank {rank}: Successfully interpolated from P3 (cubic) to P1 (linear)")

# Example 3: Interpolate expression involving coordinates
print(f"\n3. Interpolating expression f(x,y) = x^2 + y^2")

def quadratic_function(x):
    return x[0]**2 + x[1]**2

u_quad = fem.Function(V2)
u_quad.interpolate(quadratic_function)
print(f"   Rank {rank}: Interpolated quadratic expression into P2 space")

# Example 4: Interpolate vector-valued function
print(f"\n4. Interpolating vector-valued function")
V_vec = fem.functionspace(domain, ("Lagrange", 2, (domain.geometry.dim,)))

def vector_function(x):
    values = np.zeros((domain.geometry.dim, x.shape[1]))
    values[0] = np.sin(np.pi * x[0])  # x-component
    values[1] = np.cos(np.pi * x[1])  # y-component
    return values

u_vec = fem.Function(V_vec)
u_vec.interpolate(vector_function)
print(f"   Rank {rank}: Interpolated vector field into vector P2 space")
print(f"   Vector space local DOFs: {V_vec.dofmap.index_map.size_local}")

# Save functions to VTK for visualization in ParaView (ALL RANKS)
print(f"\n5. Saving functions to VTK files for ParaView:")

# Save P1 function
topology, cell_types, geometry = plot.vtk_mesh(V1)
grid1 = pv.UnstructuredGrid(topology, cell_types, geometry)
grid1.point_data["u_P1"] = u1.x.array.real
grid1.point_data["rank"] = np.full(len(u1.x.array), rank)
grid1.save(f"function_P1_rank_{rank}.vtk")
print(f"   Rank {rank}: function_P1_rank_{rank}.vtk saved")

# Save P2 function
topology, cell_types, geometry = plot.vtk_mesh(V2)
grid2 = pv.UnstructuredGrid(topology, cell_types, geometry)
grid2.point_data["u_P2"] = u2.x.array.real
grid2.point_data["rank"] = np.full(len(u2.x.array), rank)
grid2.save(f"function_P2_rank_{rank}.vtk")
print(f"   Rank {rank}: function_P2_rank_{rank}.vtk saved")

# Save P3 function
topology, cell_types, geometry = plot.vtk_mesh(V3)
grid3 = pv.UnstructuredGrid(topology, cell_types, geometry)
grid3.point_data["u_P3"] = u3.x.array.real
grid3.point_data["rank"] = np.full(len(u3.x.array), rank)
grid3.save(f"function_P3_rank_{rank}.vtk")
print(f"   Rank {rank}: function_P3_rank_{rank}.vtk saved")

# Save quadratic function
topology, cell_types, geometry = plot.vtk_mesh(V2)
grid_quad = pv.UnstructuredGrid(topology, cell_types, geometry)
grid_quad.point_data["u_quadratic"] = u_quad.x.array.real
grid_quad.point_data["rank"] = np.full(len(u_quad.x.array), rank)
grid_quad.save(f"function_quadratic_rank_{rank}.vtk")
print(f"   Rank {rank}: function_quadratic_rank_{rank}.vtk saved")

# Save vector function
topology_vec, cell_types_vec, geometry_vec = plot.vtk_mesh(V_vec)
grid_vec = pv.UnstructuredGrid(topology_vec, cell_types_vec, geometry_vec)
# For vector data, reshape and add as vectors
vec_data = u_vec.x.array.real.reshape(-1, domain.geometry.dim)
grid_vec.point_data["u_vector"] = vec_data
grid_vec.point_data["rank"] = np.full(vec_data.shape[0], rank)
grid_vec.save(f"function_vector_rank_{rank}.vtk")
print(f"   Rank {rank}: function_vector_rank_{rank}.vtk saved")

# Gather statistics from all processes
local_min = u1.x.array.real.min()
local_max = u1.x.array.real.max()
local_mean = u1.x.array.real.mean()
local_dofs = V1.dofmap.index_map.size_local

all_mins = comm.gather(local_min, root=0)
all_maxs = comm.gather(local_max, root=0)
all_means = comm.gather(local_mean, root=0)
all_dofs = comm.gather(local_dofs, root=0)

comm.Barrier()

if rank == 0:
    print("\n" + "="*70)
    print(f"Summary Across All {size} Processes:")
    print("="*70)
    print("\nFiles saved from all processes:")
    for r in range(size):
        print(f"  Rank {r}:")
        print(f"    - function_P1_rank_{r}.vtk")
        print(f"    - function_P2_rank_{r}.vtk")
        print(f"    - function_P3_rank_{r}.vtk")
        print(f"    - function_quadratic_rank_{r}.vtk")
        print(f"    - function_vector_rank_{r}.vtk")

    print("\n" + "="*70)
    print("Interpolation Summary:")
    print("  - Interpolated analytical functions into FE spaces")
    print("  - Interpolated between different degree spaces (P3 → P1)")
    print("  - Interpolated coordinate expressions")
    print("  - Interpolated vector-valued functions")
    print("  - All functions saved as VTK files (one per rank)")
    print("="*70)

    print(f"\nFunction value statistics (P1 space) per rank:")
    print(f"{'Rank':<8} {'Min':<12} {'Max':<12} {'Mean':<12} {'Local DOFs':<12}")
    print("-" * 60)
    for r in range(size):
        print(f"{r:<8} {all_mins[r]:<12.6f} {all_maxs[r]:<12.6f} {all_means[r]:<12.6f} {all_dofs[r]:<12}")

    print(f"\nGlobal statistics:")
    print(f"  Global Min: {min(all_mins):.6f}")
    print(f"  Global Max: {max(all_maxs):.6f}")
    print(f"  Global Mean (weighted): {sum(m*d for m,d in zip(all_means, all_dofs))/sum(all_dofs):.6f}")
    print(f"  Total DOFs: {V1.dofmap.index_map.size_global}")

print(f"\nRank {rank} completed successfully!")


MPI Process 0/4

DOLFINx Function Interpolation Examples on Rank 0:

Mesh partition on Rank 0:
  - Local cells: 50
  - Ghost cells: 8
  - V1 local DOFs: 30
  - V1 ghost DOFs: 13
  - V2 local DOFs: 110
  - V2 ghost DOFs: 33
  - V3 local DOFs: 240
  - V3 ghost DOFs: 61

1. Interpolating analytical function f(x,y) = sin(pi*x)*sin(pi*y)
   Rank 0: Interpolated into P1 space (Local DOFs: 30)
   Rank 0: Interpolated into P2 space (Local DOFs: 110)
   Rank 0: Interpolated into P3 space (Local DOFs: 240)

2. Interpolating from P3 space to P1 space
   Rank 0: Successfully interpolated from P3 (cubic) to P1 (linear)

3. Interpolating expression f(x,y) = x^2 + y^2
   Rank 0: Interpolated quadratic expression into P2 space

4. Interpolating vector-valued function
   Rank 0: Interpolated vector field into vector P2 space
   Vector space local DOFs: 110

5. Saving functions to VTK files for ParaView:
   Rank 0: function_P1_rank_0.vtk saved
   Rank 0: function_P2_rank_0.vtk saved
   Rank 0: function

In [4]:
%%fenicsx -np 4

import numpy as np
from mpi4py import MPI

import pyvista as pv

from dolfinx import mesh, fem, plot

# Get MPI information
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Create a simple 2D mesh (small mesh to see DOF structure clearly)
domain = mesh.create_unit_square(comm, 4, 4, mesh.CellType.triangle)

print(f"\n{'='*70}")
print(f"MPI Process {rank}/{size}")
print(f"{'='*70}")

# Create different function spaces
V1 = fem.functionspace(domain, ("Lagrange", 1))  # P1 (linear)
V2 = fem.functionspace(domain, ("Lagrange", 2))  # P2 (quadratic)

print(f"\nDOLFINx DofMap and ElementDofLayout Examples on Rank {rank}:")
print("=" * 70)

# Print partition information
print(f"\nPartition information on Rank {rank}:")
print(f"  - Local cells: {domain.topology.index_map(domain.topology.dim).size_local}")
print(f"  - Ghost cells: {domain.topology.index_map(domain.topology.dim).num_ghosts}")
print(f"  - Cell global range: [{domain.topology.index_map(domain.topology.dim).local_range[0]}, {domain.topology.index_map(domain.topology.dim).local_range[1]})")

# Example 1: Explore DofMap for P1 space
print(f"\n1. DofMap for P1 (Linear Lagrange) Space on Rank {rank}:")
print("-" * 70)

dofmap1 = V1.dofmap
index_map1 = V1.dofmap.index_map
print(f"   Total local DOFs: {index_map1.size_local}")
print(f"   Ghost DOFs: {index_map1.num_ghosts}")
print(f"   Global DOFs: {index_map1.size_global}")
print(f"   DOF global range: [{index_map1.local_range[0]}, {index_map1.local_range[1]})")
print(f"   Number of local cells: {domain.topology.index_map(domain.topology.dim).size_local}")
print(f"   DOFs per cell: {dofmap1.dof_layout.num_dofs}")

# Print DOF connectivity for each cell
print(f"\n   Cell-to-DOF connectivity (P1) on Rank {rank}:")
domain.topology.create_connectivity(domain.topology.dim, 0)  # Cell to vertex
for cell_idx in range(min(4, domain.topology.index_map(domain.topology.dim).size_local)):
    cell_dofs = dofmap1.cell_dofs(cell_idx)
    global_cell_idx = domain.topology.index_map(domain.topology.dim).local_range[0] + cell_idx
    print(f"   Local Cell {cell_idx} (Global {global_cell_idx}): DOFs = {cell_dofs}")

# Example 2: Explore DofMap for P2 space
print(f"\n2. DofMap for P2 (Quadratic Lagrange) Space on Rank {rank}:")
print("-" * 70)

dofmap2 = V2.dofmap
index_map2 = V2.dofmap.index_map
print(f"   Total local DOFs: {index_map2.size_local}")
print(f"   Ghost DOFs: {index_map2.num_ghosts}")
print(f"   Global DOFs: {index_map2.size_global}")
print(f"   DOF global range: [{index_map2.local_range[0]}, {index_map2.local_range[1]})")
print(f"   Number of local cells: {domain.topology.index_map(domain.topology.dim).size_local}")
print(f"   DOFs per cell: {dofmap2.dof_layout.num_dofs}")

# Print DOF connectivity for each cell
print(f"\n   Cell-to-DOF connectivity (P2) on Rank {rank}:")
for cell_idx in range(min(4, domain.topology.index_map(domain.topology.dim).size_local)):
    cell_dofs = dofmap2.cell_dofs(cell_idx)
    global_cell_idx = domain.topology.index_map(domain.topology.dim).local_range[0] + cell_idx
    print(f"   Local Cell {cell_idx} (Global {global_cell_idx}): DOFs = {cell_dofs}")

# Example 3: ElementDofLayout information from Basix
print(f"\n3. ElementDofLayout Information (from Basix) on Rank {rank}:")
print("-" * 70)

# P1 ElementDofLayout
dof_layout1 = V1.dofmap.dof_layout
print(f"\n   P1 ElementDofLayout:")
print(f"   - Total DOFs per cell: {dof_layout1.num_dofs}")
print(f"   - Number of entity DOFs:")
print(f"     * Vertices (dim 0): {dof_layout1.num_entity_dofs(0)}")
print(f"     * Edges (dim 1): {dof_layout1.num_entity_dofs(1)}")
print(f"     * Faces (dim 2): {dof_layout1.num_entity_dofs(2)}")

# P2 ElementDofLayout
dof_layout2 = V2.dofmap.dof_layout
print(f"\n   P2 ElementDofLayout:")
print(f"   - Total DOFs per cell: {dof_layout2.num_dofs}")
print(f"   - Number of entity DOFs:")
print(f"     * Vertices (dim 0): {dof_layout2.num_entity_dofs(0)}")
print(f"     * Edges (dim 1): {dof_layout2.num_entity_dofs(1)}")
print(f"     * Faces (dim 2): {dof_layout2.num_entity_dofs(2)}")

# Example 4: Global vs Local DOF numbering
print(f"\n4. Global vs Local DOF Numbering on Rank {rank}:")
print("-" * 70)

# Create a function and assign values based on DOF index
u1 = fem.Function(V1)
# Assign global DOF index as value (for owned DOFs)
for i in range(index_map1.size_local):
    u1.x.array[i] = index_map1.local_range[0] + i

# Ghost DOFs get special marker
for i in range(index_map1.size_local, index_map1.size_local + index_map1.num_ghosts):
    u1.x.array[i] = -1  # Mark ghost DOFs

print(f"\n   P1 Function DOF values on Rank {rank}:")
print(f"   Local DOF values (first 10): {u1.x.array[:min(10, index_map1.size_local)]}")
if index_map1.num_ghosts > 0:
    print(f"   Ghost DOF markers (first 5): {u1.x.array[index_map1.size_local:index_map1.size_local+min(5, index_map1.num_ghosts)]}")

# Show how local DOFs on cells map to global DOFs
print(f"\n   Local-to-Global DOF mapping for first 2 cells on Rank {rank}:")
for cell_idx in range(min(2, domain.topology.index_map(domain.topology.dim).size_local)):
    cell_dofs = dofmap1.cell_dofs(cell_idx)
    cell_values = u1.x.array[cell_dofs]
    global_cell_idx = domain.topology.index_map(domain.topology.dim).local_range[0] + cell_idx
    print(f"   Local Cell {cell_idx} (Global {global_cell_idx}):")
    print(f"     Local DOFs [0, 1, 2] → Global DOFs {cell_dofs}")
    print(f"     Values: {cell_values} (negative = ghost)")

# Example 5: Ghost DOF ownership information
print(f"\n5. Ghost DOF Ownership Information on Rank {rank}:")
print("-" * 70)

if index_map1.num_ghosts > 0:
    ghost_owners = index_map1.owners
    print(f"   Number of ghost DOFs: {index_map1.num_ghosts}")
    print(f"   Ghost owners (first 10): {ghost_owners[:min(10, len(ghost_owners))]}")
    print(f"   (These ranks own the ghost DOFs on this process)")
else:
    print(f"   No ghost DOFs on this rank")

# Example 6: Discontinuous Galerkin - different DOF structure
print(f"\n6. DofMap for Discontinuous Galerkin (DG) Space on Rank {rank}:")
print("-" * 70)

V_dg = fem.functionspace(domain, ("DG", 1))
dofmap_dg = V_dg.dofmap
dof_layout_dg = V_dg.dofmap.dof_layout
index_map_dg = V_dg.dofmap.index_map

print(f"   Total local DOFs: {index_map_dg.size_local}")
print(f"   Ghost DOFs: {index_map_dg.num_ghosts}")
print(f"   DOFs per cell: {dofmap_dg.dof_layout.num_dofs}")
print(f"   Number of entity DOFs:")
print(f"     * Vertices (dim 0): {dof_layout_dg.num_entity_dofs(0)}")
print(f"     * Edges (dim 1): {dof_layout_dg.num_entity_dofs(1)}")
print(f"     * Faces (dim 2): {dof_layout_dg.num_entity_dofs(2)}")

print(f"\n   Cell-to-DOF connectivity (DG1) on Rank {rank}:")
for cell_idx in range(min(4, domain.topology.index_map(domain.topology.dim).size_local)):
    cell_dofs = dofmap_dg.cell_dofs(cell_idx)
    global_cell_idx = domain.topology.index_map(domain.topology.dim).local_range[0] + cell_idx
    print(f"   Local Cell {cell_idx} (Global {global_cell_idx}): DOFs = {cell_dofs}")
print("   Note: DG elements have all DOFs inside cells (no sharing)")

# Save mesh with DOF indices for visualization (ALL RANKS)
print(f"\n7. Saving mesh and DOF information to VTK on Rank {rank}:")
print("-" * 70)

topology, cell_types, geometry = plot.vtk_mesh(V1)
grid = pv.UnstructuredGrid(topology, cell_types, geometry)
grid.point_data["Global_DOF_Index"] = u1.x.array.real
grid.point_data["MPI_Rank"] = np.full(len(u1.x.array), rank)
grid.save(f"dofmap_visualization_rank_{rank}.vtk")
print(f"   Rank {rank}: dofmap_visualization_rank_{rank}.vtk saved")

# Gather statistics from all processes
all_local_dofs = comm.gather(index_map1.size_local, root=0)
all_ghost_dofs = comm.gather(index_map1.num_ghosts, root=0)
all_local_cells = comm.gather(domain.topology.index_map(domain.topology.dim).size_local, root=0)

comm.Barrier()

if rank == 0:
    print("\n" + "="*70)
    print(f"Summary Across All {size} Processes:")
    print("="*70)

    print(f"\nDOF Distribution (P1):")
    print(f"{'Rank':<8} {'Local DOFs':<15} {'Ghost DOFs':<15} {'Local Cells':<15}")
    print("-" * 60)
    for r in range(size):
        print(f"{r:<8} {all_local_dofs[r]:<15} {all_ghost_dofs[r]:<15} {all_local_cells[r]:<15}")

    print(f"\nGlobal Totals:")
    print(f"  Total DOFs (P1): {index_map1.size_global}")
    print(f"  Total Cells: {domain.topology.index_map(domain.topology.dim).size_global}")
    print(f"  Average ghost DOFs per process: {sum(all_ghost_dofs) / size:.1f}")

    print(f"\nFiles saved from all processes:")
    for r in range(size):
        print(f"  - dofmap_visualization_rank_{r}.vtk")

    print("\n" + "="*70)
    print("Summary:")
    print("  - DofMap stores which global DOFs are on each cell")
    print("  - ElementDofLayout (from Basix) provides DOF structure info")
    print("  - P1: 1 DOF per vertex (3 DOFs per triangle)")
    print("  - P2: 1 DOF per vertex + 1 DOF per edge (6 DOFs per triangle)")
    print("  - DG: All DOFs are interior to cells (no sharing)")
    print("  - Local DOF indices map to global DOF indices")
    print("  - Ghost DOFs are copies from neighboring processes")
    print("  - Each process owns a range of global DOFs")
    print("="*70)

print(f"\nRank {rank} completed successfully!")


MPI Process 0/4

DOLFINx DofMap and ElementDofLayout Examples on Rank 0:

Partition information on Rank 0:
  - Local cells: 8
  - Ghost cells: 3
  - Cell global range: [0, 8)

1. DofMap for P1 (Linear Lagrange) Space on Rank 0:
----------------------------------------------------------------------
   Total local DOFs: 7
   Ghost DOFs: 5
   Global DOFs: 25
   DOF global range: [0, 7)
   Number of local cells: 8
   DOFs per cell: 3

   Cell-to-DOF connectivity (P1) on Rank 0:
   Local Cell 0 (Global 0): DOFs = [7 0 8]
   Local Cell 1 (Global 1): DOFs = [0 8 1]
   Local Cell 2 (Global 2): DOFs = [0 2 1]
   Local Cell 3 (Global 3): DOFs = [8 1 3]

2. DofMap for P2 (Quadratic Lagrange) Space on Rank 0:
----------------------------------------------------------------------
   Total local DOFs: 22
   Ghost DOFs: 12
   Global DOFs: 81
   DOF global range: [0, 22)
   Number of local cells: 8
   DOFs per cell: 6

   Cell-to-DOF connectivity (P2) on Rank 0:
   Local Cell 0 (Global 0): DOFs = [22